In [1]:
!pip install pyrsgis
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00


In [2]:
import os, math, numpy as np
from pyrsgis import raster, convert
from osgeo import gdal
from google.colab import drive
from copy import deepcopy
from skimage.metrics import structural_similarity as ssim
import random
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression
from PIL import Image
import numpy as np

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pop2010 = "/content/drive/MyDrive/Capstone_Data/Population_data/2010.tif"
pop2012 = "/content/drive/MyDrive/Capstone_Data/Population_data/2012.tif"
pop2014 = "/content/drive/MyDrive/Capstone_Data/Population_data/2014.tif"
pop2016 = "/content/drive/MyDrive/Capstone_Data/Population_data/2016.tif"
pop2018 = "/content/drive/MyDrive/Capstone_Data/Population_data/2018.tif"
pop2020 = "/content/drive/MyDrive/Capstone_Data/Population_data/2020.tif"

ds1, arr2010 = raster.read(pop2010,bands=1)
ds2, arr2012 = raster.read(pop2012,bands=1)
ds3, arr2014 = raster.read(pop2014,bands=1)
ds4, arr2016 = raster.read(pop2016,bands=1)
ds5, arr2018 = raster.read(pop2018,bands=1)
ds6, arr2020 = raster.read(pop2020,bands=1)

no_rows = ds1.RasterYSize
no_cols = ds1.RasterXSize

print(no_rows,no_cols)

1266 1177


In [6]:
import numpy as np
print(int(sum(np.unique(arr2010))))
print(int(sum(np.unique(arr2012))))
print(int(sum(np.unique(arr2014))))
print(int(sum(np.unique(arr2016))))
print(int(sum(np.unique(arr2018))))
print(int(sum(np.unique(arr2020))))

8541689
9376111
9952679
10558963
11209167
12347482


In [26]:
# Function to read the .tif file and return the sum of population values
def read_tif_population(file_path):
    with Image.open(file_path) as img:
        img_array = np.array(img)
        img_array[img_array == -99999] = np.nan  # Handle no-data values
        total_population = np.nansum(img_array)
    return img_array, total_population

def exportPredicted(outFileName):
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outFileName, no_cols, no_rows, 1, gdal.GDT_UInt16) # option: GDT_UInt16, GDT_Float32
    outdata.SetGeoTransform(ds6.GetGeoTransform())
    outdata.SetProjection(ds6.GetProjection())
    outdata.GetRasterBand(1).WriteArray(population_array_2030)
    outdata.GetRasterBand(1).SetNoDataValue(0)
    outdata.FlushCache()
    outdata = None

# Read the .tif files for each year and calculate the total population
file_paths = [pop2010,pop2012,pop2014,pop2016,pop2018,pop2020]
years = np.array([2010, 2012, 2014, 2016, 2018, 2020]).reshape(-1, 1)
populations = []

for file_path in file_paths:
    _, total_population = read_tif_population(file_path)
    populations.append(total_population)

model = LinearRegression()
model.fit(years, populations)

# Predict the total population for 2030
year_2030 = np.array([[2030]])
predicted_total_population_2030 = model.predict(year_2030)[0]

# Calculate the scaling factor using the actual 2020 population data
actual_population_2020 = 12_347_482  # Actual population for 2020
_, tif_population_2020 = read_tif_population(pop2020)
scaling_factor = actual_population_2020 / np.nansum(tif_population_2020)

population_array_2020, _ = read_tif_population(pop2020)
population_array_2030 = population_array_2020 * (predicted_total_population_2030 / np.nansum(population_array_2020))

# Replace NaN values back to the original no-data value
population_array_2030[np.isnan(population_array_2030)] = 0

print(int(sum(np.unique(population_array_2030))))

from sklearn.metrics import mean_squared_error

mape = (100-(16226826-15708356)/16226826*100)
print("%.3f" % mape)

exportPredicted("pop2030.tif")

15708356
96.805


In [ ]:
np.unique(arr2010, return_counts=True)
maps = []
map_years = [arr2010,arr2012,arr2014,arr2016,arr2018,arr2020]
for i in map_years:
  maps.append(i)
data = np.array(maps)
arr2030 = arr2020*1.0356*1.0335*1.0314*1.0294*1.0276*1.0261*1.0249*1.024*1.0233*1.0228
predicted_arr2020 = arr2018*1.077547
data = np.reshape(data,(data.shape[1]*data.shape[2],data.shape[0]))

input_data = data[:, :-1]
output_data = data[:, -1]

for _ in range(5):
    new_feature = np.hstack((input_data, output_data.reshape(-1, 1)))

    model = LinearRegression()
    model.fit(new_feature, output_data)

    predicted_population = model.predict(new_feature)

    input_data = np.hstack((input_data, predicted_population.reshape(-1, 1)))

    output_data = predicted_population

    print("Iteration:", _+1)
    print("Predicted population values:", predicted_population)
    print()

Iteration: 1
Predicted population values: [4695.9604 4695.9604 4695.9604 ... 1297.1979 1297.1979 1297.1979]

Iteration: 2
Predicted population values: [4703.7876 4703.7876 4703.7876 ... 1305.0242 1305.0242 1305.0242]

Iteration: 3
Predicted population values: [4703.8047 4703.8047 4703.8047 ... 1305.0254 1305.0254 1305.0254]

Iteration: 4
Predicted population values: [4703.8022 4703.8022 4703.8022 ... 1305.0214 1305.0214 1305.0214]

Iteration: 5
Predicted population values: [4703.807  4703.807  4703.807  ... 1305.0258 1305.0258 1305.0258]



In [ ]:
print(ssim(predicted_arr2020,arr2020))
arr2030 = np.reshape(output_data,(no_rows,no_cols))
print(arr2030.shape)

def exportPredicted(outFileName):
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outFileName, no_cols, no_rows, 1, gdal.GDT_UInt16)
    outdata.SetGeoTransform(ds6.GetGeoTransform())
    outdata.SetProjection(ds6.GetProjection())
    outdata.GetRasterBand(1).WriteArray(arr2030)
    outdata.GetRasterBand(1).SetNoDataValue(0)
    outdata.FlushCache()
    outdata = None


#exportPredicted("2030.tif")

0.9086479497261695
(1266, 1177)
